In [1]:
from models import *
from crawler import TiktokAnalyticAuto
import pandas as pd
import re
from datetime import datetime

In [2]:
db_setup(mode='dev')


In [3]:
# Define Constanta Path
result_path = '/home/mcimam/PersonalProject/Fyr/result/'

# Get File method
def getFile(fl_object:FileLog, base_path:str):
    path_xlsx = base_path + fl_object.name 
    df = pd.read_excel(path_xlsx, skiprows=2)
    return df 

def getFileMetadata(fl_object:FileLog, base_path:str):
    path_xlsx = base_path + fl_object.name 
    df = pd.read_excel(path_xlsx, skiprows=0, header=None)
    date_range = df.iloc[0,0]
    date_range = re.sub(r"[^0-9,~-]","",date_range)
    date_start,date_end = date_range.split("~")
    return {
        "date_start": date_start,
        "date_end":date_end
        }

In [4]:
for fi in Video.select():
    print(fi)

In [5]:
path_xlsx = result_path  + FileLog[5].name
df = pd.read_excel(path_xlsx, skiprows=0, header=None)
date_range = df.iloc[0,0]
date_range = re.sub(r"[^0-9,~-]","",date_range)
date_start,date_end = date_range.split("~")

In [16]:
f1 = FileLog[5]
d1 = getFile(f1,result_path)
md = getFileMetadata(f1,result_path)
print(d1.keys())
d1.iloc[0,:]
md

Index(['ID', 'Product Name', 'Revenue (Rp)', 'Buyers', 'Product sales',
       'Orders', 'Livestream revenue (Rp)', 'Livestream orders',
       'Livestream buyers', 'Livestream product sales',
       'Livestream product viewers', 'Users clicking livestream products',
       'Livestream product impressions', 'Livestream product clicks',
       'Livestream product CTR', 'Livestream product CO rate',
       'Video revenue (Rp)', 'Video orders', 'Video buyers',
       'Video product sales', 'Video product viewers',
       'Users clicking video products', 'Video product impressions',
       'Video product clicks', 'Video product CTR', 'Video product CO rate'],
      dtype='object')


{'date_start': '2023-06-17', 'date_end': '2023-06-17'}

In [27]:
@orm.db_session
def upsertCreator(id:int,name:str,nickname:str) -> Creator:
    """Upsert Creator Function

    Args:
        id (int): creator id
        name (str): creator name
        nickname (str): cretor nicknames

    Returns:
        Creator: Creator Objects
    """
    # upsert data
    creator = Creator.get(nickname=nickname)
    if not creator:
        creator = Creator(id=int(id))
    creator.name = str(name)
    creator.nickname = str(nickname)
    orm.flush()
    return creator

@orm.db_session
def upsertVideo(id:int,info:str) -> Video:
    """Upsert Video Function

    Args:
        id (int): id
        info (str): vide detail

    Returns:
        Video: Video Objects
    """
    # upsert data
    dt = Video.get(id=id)
    if not dt:
        dt = Video(id=int(id))
    dt.info = str(info)
    orm.flush()
    return dt

@orm.db_session
def upsertProduct(id:int,name:str) -> Product:
    """Upsert Creator Function

    Args:
        id (int): product id
        name (str): product name

    Returns:
        Product: product Objects
    """
    # upsert data
    print('--------')
    print(id)
    dt = Product.get(id=id)
    if not dt:
        dt = Product(id=int(id), name=str(name))
    dt.info = str(name)
    orm.flush()
    return dt


def convertPercentToFloat(number:str) -> float:
    """Converting Percent string to float (ex:69.69% -> 69.69)

    Args:
        number (str): string of percent

    Returns:
        float: 
    """
    if not number:
        return 0 
    return float(re.sub(r"[^0-9.eE-]", "", number))

def convertTimeToSecond(time_string:str) -> int:
    """Converiting string of hours and minutes to string

    Args:
        time_string (str): ex: "1hr 24mnt", "24 mnt", "1hr"

    Returns:
        int: _description_
    """
    if "h" in time_string:
        hour_str, minute_str = time_string.split(" ")
        hour = int(re.sub(r"[^0-9]","",hour_str))
        minute = int(re.sub(r"[^0-9]","",minute_str))
        seconds = hour * 3600 + minute * 60
    else:
        minute_str = int(time_string)
        seconds = int(re.sub(r"[^0-9]","",minute_str))* 60

    return seconds



In [29]:
@orm.db_session
def insertLiveData(dt):
    # upsert creator
    creator = upsertCreator(
        id=dt.get("Creator ID"), 
        name=dt.get('Creator'), 
        nickname=dt.get('Nickname'))
    
    #convert str to datetime
    launched_time = datetime.strptime(d.get("Launched Time"), "%Y/%m/%d/ %H:%M")
    
    # Insert Sales performace
    ld = LiveData(
        creator_id = creator,
        launched_time = launched_time,
        duration = convertTimeToSecond(dt.get("Duration")),
        # Sale performance
        revenue = int(dt.get("Revenue (Rp)")),
        product_shown = int(dt.get("Products")),
        order_created = int(dt.get("Orders Created")),
        order_paid = int(dt.get("Orders Paid")),
        unit_sales = int(dt.get("Unit Sales")),
        buyer = int(dt.get("Buyers")),
        avg_price = int(dt.get("Average Price (Rp)")),
        co_rate= convertPercentToFloat(dt.get("CO Rate")),
        impression =int(dt.get("Product Impressions")),
        product_click=int(dt.get("Product Clicks")),
        ctr = convertPercentToFloat(dt.get("CTR")),
        # Live Peformance
        viewer = int(dt.get("Viewers")),
        view = int(dt.get("Views")),
        acu = int(dt.get("ACU")),
        pcu = int(dt.get("PCU")),
        avg_view_duration = int(dt.get("Avg. Viewing Duration")),
        comment = int(dt.get("Comments")),
        share = int(dt.get("Shares")),
        like = int(dt.get("Likes")),
        new_follower = int(dt.get("New Followers"))
    )        
    orm.flush()        
    
@orm.db_session
def insertProductData(dt):
    product = upsertProduct(
        id = dt['ID'],
        name = dt.get('Product Name')
    )
    pd = ProductData(
            product = product,
            revenue = int(dt.get('Revenue (Rp)')),
            buyer = int(dt.get('Buyers')),
            sale = int(dt.get('Product sales')),
            order = int(dt.get('Orders')),
            # Livestream so
            live_buyer = int(dt.get('Livestream buyers')),
            live_sale = int(dt.get('Livestream product sales')),
            live_viewer = int(dt.get('Livestream product viewers')),
            live_click = int(dt.get('Livestream product clicks')),
            live_impression = int(dt.get('Livestream product impressions')),
            live_ctr = convertPercentToFloat(dt.get('Livestream product CTR')),
            live_co = convertPercentToFloat(dt.get('Livestream product CO rate')),
            # Video source
            video_buyer = int(dt.get('Video buyers')),
            video_sale = int(dt.get('Video product sales')),
            video_viewer = int(dt.get('Video product viewers')),
            video_click = int(dt.get('Video product clicks')),
            video_impression = int(dt.get('Video product impressions')),
            video_ctr = convertPercentToFloat(dt.get('Video product CTR')),
            video_co = convertPercentToFloat(dt.get('Video product CO rate'))

    )
    
for i,d in d1.iterrows():
    # insertLiveData(d)
    # insertProductData(d)

--------
1729384826829178211


ValueError: Attribute ProductData.id is required

In [55]:
convertTimeToSecond(d["Duration"])

6840